# Prediction of Hospital Readmissions

This notebook's goal is to reproduce the claims from Zebin and Chaussalet's paper, 'Design and implementation of a deep recurrent model for prediction of readadmission in urgent care using electronic health records'.



## Claims

When predicting ICU readmissions:

1. LSTM+CNN produced higher accuracy than logistic regression, random forest, and SVM.
2. LSTM+CNN produced higher precision than logistic regression, random forest, and SVM.
3. LSTM+CNN produced higher recall than logistic regression and SVM

In [ ]:
import numpy as np
import pandas as pd
import torch
import rdkit
import sklearn
import networkx
import tqdm
#import pyhealth
import matplotlib.pyplot as plt
import csv
import os

In [ ]:
patients = pd.read_csv('./mimic-iii/PATIENTS.csv')
patients.head()

In [ ]:
admissions = pd.read_csv('./mimic-iii/ADMISSIONS.csv')
admissions.head()

In [ ]:
transfers = pd.read_csv('./mimic-iii/TRANSFERS.csv')
transfers.head()

## Example

Patient with `SUBJECT_ID = 250`

In [ ]:
patients[patients['SUBJECT_ID'] == 250]

In [ ]:
admissions[admissions['SUBJECT_ID'] == 250]

In [ ]:
transfers[transfers['SUBJECT_ID'] == 250]

## Create Dataset

SUBJECT_ID exmaples  291, 283, 250

In [ ]:
patients['DOB'] = pd.to_datetime(patients['DOB'], errors='coerce')
patients['DOD'] = pd.to_datetime(patients['DOD'], errors='coerce')
patients['DOD_HOSP'] = pd.to_datetime(patients['DOD_HOSP'], errors='coerce')
patients['DOD_SSN'] = pd.to_datetime(patients['DOD_SSN'], errors='coerce')

In [ ]:
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'], errors='coerce')
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'], errors='coerce')
admissions['DEATHTIME'] = pd.to_datetime(admissions['DEATHTIME'], errors='coerce')

In [ ]:
transfers['INTIME'] = pd.to_datetime(transfers['INTIME'], errors='coerce')
transfers['OUTTIME'] = pd.to_datetime(transfers['OUTTIME'], errors='coerce')

In [ ]:
dataset = admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME']]
dataset = pd.merge(dataset, patients[['SUBJECT_ID','GENDER','DOB','DOD','DOD_HOSP','DOD_SSN','EXPIRE_FLAG']], how='left', on='SUBJECT_ID')
#dataset = dataset.drop('DEATHTIME', axis=1)


In [ ]:
print(len(dataset[dataset['DEATHTIME'].notnull()].SUBJECT_ID.unique()))


In [ ]:
# Remove under 18
dataset['AGE'] = ((pd.to_datetime(dataset['ADMITTIME']).dt.date - pd.to_datetime(dataset['DOB']).dt.date) / np.timedelta64(1, 'Y')).astype(int)
dataset = dataset[dataset['AGE'] >= 18]

In [ ]:
print(dataset.shape)
dataset.head(5)


## Test

In [ ]:
dataset = pd.merge(dataset, transfers[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'EVENTTYPE', 'PREV_CAREUNIT', 'CURR_CAREUNIT', 'INTIME', 'OUTTIME', 'LOS']], how='left', on=['SUBJECT_ID','HADM_ID'])

In [ ]:

# test_dataset_admissions = dataset
# test_trasfers = transfers[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'EVENTTYPE', 'PREV_CAREUNIT', 'CURR_CAREUNIT', 'INTIME', 'OUTTIME', 'LOS']]

In [ ]:
# test_dataset = test_dataset_admissions.merge(test_trasfers, how='left', on=['SUBJECT_ID','HADM_ID'])

In [ ]:
dataset.head()

In [ ]:
mortality_test = dataset.copy()

In [ ]:
dataset.shape

In [ ]:
# died in hospital
expired = dataset[~dataset['DOD_HOSP'].isnull()]


In [ ]:
expired.head(5)

In [ ]:
# create normalized transfer times for comparison to DOD
expired.loc[:,'INTIME_NORM'] = expired.loc[:,'INTIME'].dt.normalize()
expired.loc[:,'OUTTIME_NORM'] = expired.loc[:,'OUTTIME'].dt.normalize()

In [ ]:
np.sum(expired['DOD'] == expired['INTIME_NORM'])

In [ ]:
# died in icu if conditions met
expired_in_icu = expired[(expired['EVENTTYPE']=='discharge') & (~expired['PREV_CAREUNIT'].isnull()) & (expired['INTIME_NORM'] == expired['DOD'])]

In [ ]:
expired_in_icu.head()

In [ ]:
# make mask of people who died in icu for use on HADM_ID
print(dataset.shape)
hadm_id_unique = expired_in_icu['HADM_ID'].unique()
dataset = dataset[~np.in1d(dataset['HADM_ID'], hadm_id_unique )]
print(dataset.shape)
# out = chartEvents[np.in1d(chartEvents['ITEMID'], transferID )]

In [ ]:
print('Total ICU stays:\t', len(dataset.HADM_ID.unique()))
print('Total Patients: \t', len(dataset.SUBJECT_ID.unique()))

## Mortality Test

In [ ]:
mortality_test.head()

In [ ]:
mortality = mortality_test.DEATHTIME.notnull() & ((mortality_test.ADMITTIME <= mortality_test.DEATHTIME) & (mortality_test.DISCHTIME >= mortality_test.DEATHTIME))
mortality_test['MORTALITY_INHOSPITAL'] = mortality.astype(int)

mortality = mortality_test.DEATHTIME.notnull() & ((mortality_test.INTIME <= mortality_test.DEATHTIME) & (mortality_test.OUTTIME >= mortality_test.DEATHTIME))
mortality_test['MORTALITY_INUNIT'] = mortality.astype(int)

In [ ]:
dt = mortality_test[mortality_test['DEATHTIME'].notnull()]
print(len(dt.SUBJECT_ID.unique()))
dod = mortality_test[mortality_test['DOD_HOSP'].notnull()]
print(len(dod.SUBJECT_ID.unique()))

print('DEATHTIME', np.sum(mortality_test.DEATHTIME.notnull()))
print('DOD_HOSP', np.sum(mortality_test.DOD_HOSP.notnull()))